# Tutorial 12: Bottleneck Experiments 瓶颈实验

This tutorial walks you through the process of *running the bottleneck experiments*. The bottleneck experiment, depicted in Fig. 1, is intended to illustrate the dynamics of traffic in a bottleneck. In particular, our bottleneck is intended to imitate the dynamics of traffic on the San Francisco-Oakland Bay Bridge, where fifteen lanes narrow to eight to five. In our bottleneck, we have N * 4 lanes narrow to N * 2 to N, where N is a scaling factor that can be used to increase the number of lanes. As demonstrated in Fig. 2, this bottleneck has a phenomenon known as *capacity drop*: as the number of vehicles flowing into the system increases the number of exiting vehicles initially increases. However, above a critical flow of entering vehicles, the output of the system starts to decrease as congestion forms. Just like in the San Francisco-Oakland Bay Bridge, there is a configurable toll booth and a set of traffic lights that can be used to control the inflow. Each of the merges is implemented as a zipper merge, in which the vehicles merge as late as possible. 

本教程将指导您完成“运行瓶颈实验”的过程。图1所示的瓶颈实验旨在说明瓶颈中的流量动态。特别地，我们的瓶颈旨在模拟旧金山-奥克兰海湾大桥上的交通动态，那里有15条车道窄至8至5条。在我们的瓶颈中，我们有N * 4条窄至N * 2到N的车道，其中N是一个比例因子，可以用来增加车道的数量。如图2所示，这一瓶颈出现了*容量下降*的现象:随着进入系统的车辆数量的增加，最初退出系统的车辆数量也随之增加。然而，在进入车辆的临界流量之上，系统的输出开始以拥塞的形式减少。就像在旧金山-奥克兰海湾大桥，有一个可配置的收费站和一组红绿灯，可以用来控制流入。每一个合并都是作为一个zippers合并来实现的，在这个过程中，车辆合并的时间越晚越好。

<center>
<img src="img/bottleneck.png">

Figure 1. A visual of the bottleneck that is modeled on the San Francisco-Oakland bay bridge. There are two bottlenecks, the first cuts the number of lanes in half and the second cuts the number of lanes in half again.图1所示。以旧金山-奥克兰海湾大桥为模型的瓶颈的视觉效果。这里有两个瓶颈，第一个是车道数减半，第二个是车道数减半。</center>

<center>
<img src="img/capacity_curve.png">
</center>
<center>
Figure 2. The inflow-outflow relationship for the bottleneck depicted in Fig. 1. As the inflow increases, the outflow steadily ticks up, but above a critical inflow the outflow suddenly ticks down and remains at that value. This phenomenon is known as *capacity drop*. The shaded area indicates 1 std deviation around the outflow.图2。图1所示的瓶颈的流入流出关系。随着流入的增加，流出稳步上升，但在临界流入以上，流出突然下降，并保持在该值。这种现象称为“容量下降”。阴影部分表示流出口周围有1个std偏差。
</center>


The remainder of this tutorial is organized as follows:

* Section 1 introduces the basic configurable parameters of the traffic light environment.介绍了交通灯环境的基本配置参数。
* Section 2 shows how to configure the toll booth.展示如何配置收费站。
* Section 3 shows how to use the traffic lights to perform ramp metering.演示如何使用交通灯来执行匝道测量。
* Section 4 introduces some autonomous vehicles into the system and describes the Markov Decision Process that the bottleneck comes pre-configured with. 在系统中引入了一些自动驾驶车辆，并描述了瓶颈预先配置的马尔可夫决策过程。

## 1. Configuring the Environment

Here we describe the different basic configurable parameters of the bottleneck environment. First, we import the necessary classes and parameters to run the environment. We will highlight the effects of changing certain parameters.
在这里，我们描述了瓶颈环境的不同基本可配置参数。首先，我们导入必要的类和参数来运行环境。我们将强调改变某些参数的影响。

In [ ]:
# Import all of the necessary pieces of Flow to run the experiments
from flow.core.params import SumoParams, EnvParams, NetParams, InitialConfig, \
    InFlows, SumoLaneChangeParams, SumoCarFollowingParams
from flow.core.params import VehicleParams
from flow.core.params import TrafficLightParams

from flow.networks.bottleneck import BottleneckNetwork
from flow.controllers import SimLaneChangeController, ContinuousRouter
from flow.envs.bottleneck import BottleneckEnv
from flow.core.experiment import Experiment

import logging

def run_exp(flow_rate,
            scaling=1,
            disable_tb=True,
            disable_ramp_meter=True,
            n_crit=1000,
            feedback_coef=20):
    # Set up SUMO to render the results, take a time_step of 0.5 seconds per simulation step
    sim_params = SumoParams(
        sim_step=0.5,
        render=True,
        overtake_right=False,
        restart_instance=False)

    vehicles = VehicleParams()

    # Add a few vehicles to initialize the simulation. The vehicles have all lane changing enabled, 
    # which is mode 1621
    vehicles.add(
        veh_id="human",
        lane_change_controller=(SimLaneChangeController, {}),
        routing_controller=(ContinuousRouter, {}),
        car_following_params=SumoCarFollowingParams(
            speed_mode=25,
        ),
        lane_change_params=SumoLaneChangeParams(
            lane_change_mode=1621,
        ),
        num_vehicles=1)

    # These are additional params that configure the bottleneck experiment. They are explained in more
    # detail below.
    additional_env_params = {
        "target_velocity": 40,
        "max_accel": 1,
        "max_decel": 1,
        "lane_change_duration": 5,
        "add_rl_if_exit": False,
        "disable_tb": disable_tb,
        "disable_ramp_metering": disable_ramp_meter,
        "n_crit": n_crit,
        "feedback_coeff": feedback_coef,
    }
    # Set up the experiment to run for 1000 time steps i.e. 500 seconds (1000 * 0.5)
    env_params = EnvParams(
        horizon=1000, additional_params=additional_env_params)

    # Add vehicle inflows at the front of the bottleneck. They enter with a flow_rate number of vehicles 
    # per hours and with a speed of 10 m/s
    inflow = InFlows()
    inflow.add(
        veh_type="human",
        edge="1",
        vehsPerHour=flow_rate,
        departLane="random",
        departSpeed=10)

    # Initialize the traffic lights. The meanings of disable_tb and disable_ramp_meter are discussed later.
    traffic_lights = TrafficLightParams()
    if not disable_tb:
        traffic_lights.add(node_id="2")
    if not disable_ramp_meter:
        traffic_lights.add(node_id="3")

    additional_net_params = {"scaling": scaling, "speed_limit": 23}
    net_params = NetParams(
        inflows=inflow,
        additional_params=additional_net_params)

    initial_config = InitialConfig(
        spacing="random",
        min_gap=5,
        lanes_distribution=float("inf"),
        edges_distribution=["2", "3", "4", "5"])

    flow_params = dict(
        exp_tag='bay_bridge_toll',
        env_name=BottleneckEnv,
        network=BottleneckNetwork,
        simulator='traci',
        sim=sim_params,
        env=env_params,
        net=net_params,
        veh=vehicles,
        initial=initial_config,
        tls=traffic_lights,
    )

    # number of time steps
    flow_params['env'].horizon = 1000
    exp = Experiment(flow_params)

    # run the sumo simulation
    _ = exp.run(1)

### The effects of scaling 缩放的影响
Setting scaling to values greater than 1 leads to the number of lanes increasing. Scaling=1 means we start with four lanes, scaling=2 leads to us starting wth 8 lanes, etc.
将比例设置为大于1的值会导致车道数增加。比例=1表示我们从4条车道开始，比例=2表示我们从8条车道开始，等等。

Fig. 3 depicts the effect of scaling on the bottleneck.
图3描述了缩放对瓶颈的影响。

<center>
<img src="img/bottleneck_scaling_1.png">
<img src="img/bottleneck_scaling_2.png">

Figure 3. The effects of scaling on the bottleneck. If we set scaling to 1, the number of lanes goes 4 to 2 to 1. If we set scaling to 2, the number of lanes goes 8 to 4 to 2. This pattern continues as we increase the scaling values.
    图3。扩展对瓶颈的影响。如果我们把比例设为1，车道数就变成了4到2到1。如果我们将比例设置为2，车道的数量就会从8条变为4条，再变为2条。当我们增加缩放值时，该模式将继续。
</center>

**scaling=1**

In [ ]:
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=True)

**scaling=2**

In [ ]:
run_exp(flow_rate=1000, scaling=2, disable_tb=True, disable_ramp_meter=True)

### The effects of inflow 流入的影响
Increasing the inflow rate into the system controls whether congestion sets in or not. For values approximately less than 1000, congestion rarely sets in. As the value of the inflow increases, congestion becomes more likely. Around an inflow of around 1600 congestion occurs with high certainty.

增加流入系统的流量可以控制是否出现拥塞。对于大约小于1000的值，拥塞很少出现。随着流入流量的增加，出现拥塞的可能性增大。大约在1600年左右，拥塞肯定会发生。

In [ ]:
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=True)

In [ ]:
run_exp(flow_rate=1400, scaling=1, disable_tb=True, disable_ramp_meter=True)

In [ ]:
run_exp(flow_rate=2400, scaling=1, disable_tb=True, disable_ramp_meter=True)

## Adding a tollbooth 添加一个过路收费亭
On the segment indicated in Fig. 4, we have set up a series of rules that allow users to mimic the effect of a tollbooth. If *disable_tb=False*, cars that approach the toll-booth have their color changed to blue to indicate that they are in the toll-booth region. Their lane changing is disabled. As they come to the front of the toll-booth, we sample from a gaussian to determine how long they should be held at the toll booth. The holding process is imitated by a red light that remains red for the duration of the holding time. As indicated in the figure, the outer lanes are fast-trak lanes; their mean holding time is set to be lower than the holding time of the other toll-booth lanes. For the exact values of the holding parameters, please refer to the *BottleneckEnv* class in *flow/envs/bottleneck.py*

在图4所示的路段，我们设置了一系列规则，允许用户模仿收费站的效果。如果disable_tb=False，接近收费站的车辆将其颜色更改为蓝色，以表明它们在收费站区域。他们的变道被禁用了。当他们来到收费亭前面时，我们从高斯分布中取样，以确定他们应该在收费亭停留多长时间。在保持时间内保持红色的红灯模拟保持过程。如图所示，外围车道为快速轨道车道;他们的平均等待时间设定为低于其他收费站的等待时间。有关保持参数的确切值，请参考flow/envs/ k.py中的瓶颈类

<img src="img/bottleneck_guide.png">

<center>Figure 4: A bottleneck with both the toll and the ramp meter enabled图4:启用收费和匝道收费表的瓶颈</center>

In [ ]:
run_exp(flow_rate=1000, scaling=1, disable_tb=False, disable_ramp_meter=True)

## Adding a ramp meter 添加斜坡流量计
As indicated in Fig. 5, we also have a feedback ramp meter that can be used to apply control to the bottleneck. The ramp meter runs the ALINEA algorithm for metering control; in short, the algorithm controls how long the lights are red for as a function of the congestion. If the number of vehicles in section 4, shown in Fig. 6 exceed a critical value then the red time of the light begins to increase so as to restrict the inflow and allow congestion to dissipate. If it decreases below a critical value then the red time of the light begins to decrease. Note that vehicles approaching the light are colored blue and their lane changing is disabled until they pass through the light.
如图5所示，我们还有一个反馈匝道仪表，可用于对瓶颈进行控制。斜坡式仪表运行ALINEA算法进行计量控制;简而言之，算法根据拥塞情况来控制红灯的时间。如果图6所示的第四段车辆数量超过了临界值，则红灯时间开始增加，从而限制流入，使拥堵得以消散。如果它下降到一个临界值以下，那么红光时间开始减少。请注意，接近红绿灯的车辆都是蓝色的，在通过红绿灯之前，它们的变道是禁用的。

<img src="img/segment_4_highlight.png">

<center>Figure 4: The section whose number of vehicles we use to decide how long the red light is on for.图4:我们用来决定红灯亮多长时间的车辆数量部分。</center>

We highlight a few of the parameters that can be used to modify the ALINEA algorithm:我们强调了一些参数，可以用来修改ALINEA算法:

### n_crit:
*n_crit* sets the number of vehicles around which we try to stabilize the system. If the number of vehicles in section 4 increases above n_crit the red time will begin to increase. If *n_crit* is set too low than the flow will not be maximized as the traffic lights will be too aggressive in restricting inflow; conversely if n_crit is set too high then the traffic lights will not be aggressive enough and the congestion will never dissipate.
n_crit设置我们试图稳定系统周围的车辆数量。如果第4段的车辆数量增加到n_crit以上，则红色时间开始增加。如果n_crit设置得过低，流量将不会达到最大，因为交通灯在限制流量时会过于激进;相反地，如果n_crit设置得太高，那么交通灯就不会有足够的侵略性，交通拥堵也永远不会消失。

In [ ]:
## An example of a good value of n_crit
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=8)

In [ ]:
## An example where n_crit is way too low
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=2)

In [ ]:
## An example where n_crit is way too high
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=30)

### feedback_coef:
*feedback_coef* sets the aggressiveness with which the red light time changes in response to the number of vehicles in section 4 not being at *n_crit*. High values cause the total red light time to shift very aggressively, low values cause it to react more slowly. You can play with the values below and see the effects

feedback_coef设置红灯时间随第4节中未达到n_crit的车辆数量变化的攻击性。高数值导致红灯总时间变化非常快，低数值导致反应更慢。您可以使用下面的值并查看效果

In [ ]:
## An example of a relatively stable feedback value
run_exp(flow_rate=1000, scaling=1, disable_tb=False, disable_ramp_meter=False, n_crit=8, feedback_coef=20)

In [ ]:
## An example of a feedback value that will cause the red time to swing wildly
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=8, feedback_coef=200)

In [ ]:
## An example of a feedback value that will cause the red time to change too slowly
run_exp(flow_rate=1000, scaling=1, disable_tb=True, disable_ramp_meter=False, n_crit=8, feedback_coef=1)

For a reference on all of the different parameters that can be used to modify the ALINEA algorithm, please refer to the documentation in *BottleneckEnv* in *flow/envs/bottleneck.py*

对于所有可以用来修改ALINEA算法的不同参数，请参考 flow/envs/bottleneck.py

## Instantiating a reinforcement learning experiment 实例化一个强化学习实验

While there are many different ways you might use autonomous vehicles (AVs) to try to prevent the effect of a capacity drop, here we demonstrate the particular control scheme used in "Lagrangian control through deep-rl: Applications to bottleneck decongestion" by E. Vinitsky, K. Parvate et. al. 

虽然有许多不同的方法可以使用自动驾驶车辆(AVs)来试图防止容量下降的影响，但在这里，我们将演示E. Vinitsky, K. Parvate等人在“通过deep-rl的拉格朗日控制:用于瓶颈解充血的应用”中使用的特殊控制方案。

The code referenced is in 引用的代码在 *examples/exp_configs/rl/singleagent/singleagent_bottleneck.py*.

We insert a flow of autonomous vehicles as a fraction of the total flow of the system. Due to randomness in the inflow, the number of AVs in the system varies.
In this scheme, all of the AVs are controlled by a single controller. However, because neural network controllers necessarily take in a fixed sized input and have a fixed size output, we come up with a state parametrization and action parametrization that can handle the varying number of vehicles.

我们插入一个自动车辆流作为系统总流的一部分。由于流入的随机性，系统中AVs的数量是变化的。

在该方案中，所有的AVs都由一个控制器控制。然而，由于神经网络控制器必须接受固定大小的输入并具有固定大小的输出，因此我们提出了一种状态参数化和动作参数化方法来处理不同数量的车辆。

To create a fixed size set of states, the entire bottleneck is divided into segments, depicted in white in Fig. 7, and in each of those segments we return aggregate statistics about the speed, number, and density of both the vehicles and AVs in each of the lanes in the segment. For the actions, in each of the action-segments, depicted in ref in Fig. 7 we allow the controller to increase or decrease the speed limit of all the AVs in each lane in that segment.  

创建一个固定大小的州,整个瓶颈分为段,白色中描述在图7中,在每一段我们返回聚合速度的统计信息,数字,和密度的各车道的车辆和AVs段。对于图7中ref所示的每个动作段中的动作，我们允许控制器提高或降低该段中每个泳道中所有AVs的速度限制。

<img src="img/bottleneck_control_scheme.png">

<center>Figure 7: A depiction of the scheme used to create a fixed size state and action space for the bottleneck. Segments in white correspond to the division of the bottleneck for states, and segments in red for the division into action. The key at the bottom left indicates the actual information used in the state space for each lane. The numbers for each lane indicate the values that the information would take on for that lane.图7:用于为瓶颈创建固定大小的状态和操作空间的方案的描述。白色的段表示瓶颈状态的划分，红色的段表示动作的划分。左下角的键表示每个泳道的状态空间中使用的实际信息。每个泳道的数字表示该信息将为该泳道提供的值。</center>

### Code
Note, for the following code to run it is necessary to have RLlib installed. It should be installed if you have completed the [Flow setup instructions](https://flow.readthedocs.io/en/latest/flow_setup.html#local-installation-of-flow).

注意，要运行以下代码，必须安装RLlib。如果您已经完成了流程设置说明，则应该安装它。

The relevant parameters are:相关参数为:
- symmetric: If true, the same action is applied to every lane in each segment
对称:如果为真，则对每个段中的每个泳道应用相同的操作
- controlled_segments: A list of tuples describing wheher a segment is controlled, how many divisions each segment is broken up into, and the name of the segment. For example, controlled_segments = `[("1", 1, False), ("2", 2, True), ("3", 2, True), ("4", 2, True), ("5", 1, False)]` indicates that segment 1 is uncontrolled, segment 2 is controlled and is divided into two pieces, etc.
controlled_segments:一个元组列表，描述一个segment是否被控制，每个segment有多少个division，以及segment的名字。例如，controlled_segment = [("1"， 1, False)， ("2"， 2, True)， ("3"， 2, True)， ("4"， 2, True)， ("5"， 1, False)]表示段1是不受控的，段2是受控的，分为两部分，等等。
- observed segments: : A list of tuples describing wheher a segment is observed, how many divisions each segment is broken up into, and the name of the segment. For example, observed_segments = `[("1", 1, False), ("2", 2, True), ("3", 2, True), ("4", 2, True), ("5", 1, False)]` indicates that segment 1 is uncontrolled, segment 2 is controlled and is divided into two pieces, etc.
观察段:一个元组列表，描述一个段是否被观察，每个段被分成多少个部分，以及该段的名称。例如，observed_segment = [("1"， 1, False)， ("2"， 2, True)， ("3"， 2, True)， ("4"， 2, True)， ("5"， 1, False)]表示段1是不受控的，段2是受控的，分为两部分，等等。
- reset_inflow: If true, the inflow is randomized for every rollout.
reset_flows:如果为真，则对每个rollout随机分配流入。
- inflow_range: This contains the range of inflows from which inflows are uniformly sampled if reset_inflow is true.
inflow_range:它包含流入的范围，如果reset_flows为真，则从该范围均匀采样流入。

## Multi-agent reinforcement learning experiment
### In progress